In [1]:
import numpy as np
import pandas as pd

In [184]:
df = pd.read_excel(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/HIMSS/dbo_HAEntity_2017.xlsx')
len(df)

65534

In [120]:
65534-65145

389

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65534 entries, 0 to 65533
Data columns (total 57 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   HAEntityId                     65534 non-null  int64  
 1   SurveyId                       65534 non-null  int64  
 2   ParentId                       63393 non-null  float64
 3   UniqueId                       65534 non-null  int64  
 4   EntityNo                       65534 non-null  int64  
 5   Name                           65534 non-null  object 
 6   County                         64996 non-null  object 
 7   HAEntityTypeId                 65534 non-null  int64  
 8   HAEntityType                   65534 non-null  object 
 9   CBSA                           54206 non-null  object 
 10  MedicareNumber                 17397 non-null  object 
 11  Address1                       65145 non-null  object 
 12  Address2                       30219 non-null 

In [122]:
# Filter columns (Eliminate columns with too much nan values)
df = df[['HAEntityId', 'UniqueId', 'Name',
       'County', 'HAEntityTypeId', 'CBSA', 
       'Address1', 'Address2', 'City', 'State', 'Zip', 'Type',
       'YearOpened', 'OwnershipStatus', 
       'FreeStanding', 'SameISSystem', 
       'NofPhysicians', 
       'IsImaging', 'Latitude', 'Longitude', 'IsIDSACOClassified',
       'IsIDSPCMHCertified', 'IsIDSPlanningACO', 'IsIDSPlanningPCMH','PhysicianExtenders',
       'IsNICUPresent', 'ParentId', 'SurveyId', 'MedicareNumber','Phone', 'HAEntityType']]

In [123]:
# Clean data types
df['HAEntityId'] = df['HAEntityId'].astype(str)
df['UniqueId'] = df['UniqueId'].astype(str)
df['EntityNo'] = df['EntityNo'].astype(str)
df['HAEntityTypeId'] = df['HAEntityTypeId'].astype(str)
df['ParentId'] = df['ParentId'].astype(str)
df['SurveyId'] = df['SurveyId'].astype(str)


In [38]:
df.describe()

,YearOpened,NofPhysicians,Latitude,Longitude,PhysicianExtenders
count,25445.000000,45304.000000,63645.000000,63645.000000,41087.000000
mean,1995.088544,5.575556,38.553618,-90.094802,1.217538
std,33.852837,26.697272,4.894601,14.705231,8.758739
min,1040.000000,0.000000,7.172555,-166.212100,0.000000
25%,1993.000000,1.000000,35.209043,-95.997371,0.000000
50%,1998.000000,2.000000,39.393115,-86.455830,0.000000
75%,2007.000000,5.000000,41.785904,-80.045799,1.000000
max,6088.000000,2300.000000,68.540500,111.924956,1560.000000


##### Find County 
We need to find County by using zip codes.

Some rows are missing zip codes. We need to find the zip codes by city and state information

In sum, we need to (1) find missing zip codes using city and state information, and (2) find county using zip codes

In [137]:
city_zip_df = df[df['Zip'].isna()][['City','State']].drop_duplicates()

,City,State,County
0,Charlotte,NC,Mecklenburg County
2,Springfield,MO,Greene County
3,Greenville,NC,Pitt County
4,Buffalo,NY,Erie County
7,Springfield,OR,Lane County
...,...,...,...
384,Spokane,WA,Spokane County
385,Burbank,CA,Los Angeles County
386,Danville,PA,Montour County
387,Toledo,OH,Lucas County


In [130]:
city_zip_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 0 to 388
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    306 non-null    object
 1   State   306 non-null    object
dtypes: object(2)
memory usage: 7.2+ KB


In [ ]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

In [101]:
city_list = [i for i in city_zip_df['City']]
state_list = [i for i in city_zip_df['State']]

new_list = []
multi_dict = {}

for i in range(len(city_list)):
    if i%50==0:
        print(i)
    try:
        res = search.by_city_and_state(city_list[i], state_list[i])
        county_set = set([r.county for r in res])
        if len(county_set)==1:
            new_list.append(list(county_set)[0])
        else:
            new_list.append(np.nan)
            multi_dict[str(city_list[i])+"_"+str(state_list[i])] = list(county_set)
            
    except Exception as e:
        print(e)
        print('\n')
        print(city_list[i], state_list[i])
        
city_zip_df['County'] = new_list

0
50
100
150
200
250
300


In [144]:
# Make full data set
missing_zip_df = pd.merge(df[df['Zip'].isna()].drop(columns=['County']), city_zip_df, on=['City','State'])

df = pd.concat([missing_zip_df, df[~df['Zip'].isna()]], join='inner')

- Try once more

In [152]:
city_zip_df = df[df['County'].isna()][['City','State']].drop_duplicates()

In [153]:
city_zip_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 11 to 64878
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    144 non-null    object
 1   State   144 non-null    object
dtypes: object(2)
memory usage: 3.4+ KB


In [154]:
city_list = [i for i in city_zip_df['City']]
state_list = [i for i in city_zip_df['State']]

new_list = []
multi_dict = {}

for i in range(len(city_list)):
    if i%50==0:
        print(i)
    try:
        res = search.by_city_and_state(city_list[i], state_list[i])
        county_set = set([r.county for r in res])
        if len(county_set)==1:
            new_list.append(list(county_set)[0])
        else:
            new_list.append(np.nan)
            multi_dict[str(city_list[i])+"_"+str(state_list[i])] = list(county_set)
            
    except Exception as e:
        print(e)
        print('\n')
        print(city_list[i], state_list[i])
        
city_zip_df['County'] = new_list

0
50
100


In [193]:
list(set(df[df['County'].isna()].columns) & set(city_zip_df.columns))

['County', 'State', 'City']

In [194]:
# Make full data set
missing_county_df = pd.merge(df[df['County'].isna()].drop(columns=['County']), city_zip_df, on=['City','State'])

df = pd.concat([missing_county_df, df[~df['County'].isna()]], join='inner')

In [217]:
df = df[['HAEntityId', 'UniqueId', 'Name', 
         'State', 'County', 'HAEntityType', 'Type', 'YearOpened',
         'OwnershipStatus', 'FreeStanding', 'SameISSystem', 'NofPhysicians',
         'IsImaging', 'Latitude', 'Longitude', 'IsIDSACOClassified',
         'IsIDSPCMHCertified', 'IsIDSPlanningACO', 'IsIDSPlanningPCMH',
         'PhysicianExtenders', 'IsNICUPresent', 'ParentId', 'SurveyId',
         'MedicareNumber', 'Phone', 'City', 'HAEntityTypeId','CBSA',
         'Address1', 'Address2', 'Zip', ]]

In [221]:
df.to_excel(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/HIMSS/dbo_HAEntity_2017_county.xlsx', index=False)